<a href="https://colab.research.google.com/github/kavyajeetbora/recipe_recommender/blob/master/notebooks/food_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=04bbf8b511e2d01f4aeb43bf1d8c0c78086fb96eaa4e7180a9a2eec84e28dfc3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [42]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
import torch
from tqdm.notebook import tqdm
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read the data
nutrition - calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) ,carbohydrates (PDV)

In [4]:
%%time
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/food_recommender/data_file/RAW_recipes.csv')
df = df.reset_index(drop=True)
df.head()

CPU times: user 4.35 s, sys: 602 ms, total: 4.95 s
Wall time: 9.92 s


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


## Transform the data

In [5]:
## Split the nutrition column to separate columns
df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df.nutrition.str.split(",",expand=True)

## Replace [] brackets after split
df['calories'] =  df['calories'].apply(lambda x: x.replace('[',''))
df['carbohydrates (PDV)'] =  df['carbohydrates (PDV)'].apply(lambda x: x.replace(']',''))

## Convert the datatypes from str to float
df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']].astype('float')

## Drop unnecessary columns
df.drop(['id', 'contributor_id','submitted','tags','nutrition'], axis=1,inplace = True)

In [6]:
df.dtypes

name                    object
minutes                  int64
n_steps                  int64
steps                   object
description             object
ingredients             object
n_ingredients            int64
calories               float64
total fat (PDV)        float64
sugar (PDV)            float64
sodium (PDV)           float64
protein (PDV)          float64
saturated fat (PDV)    float64
carbohydrates (PDV)    float64
dtype: object

In [7]:
df.head()

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,arriba baked winter squash mexican style,55,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,30,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,130,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,45,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,190,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


## Cleaning the ingredients column

In [8]:
%%time
## First convert the string representation of list to list
df['ingredients'] = df['ingredients'].str.strip("[]").str.replace("'","") #.str.split(",")

CPU times: user 538 ms, sys: 48.9 ms, total: 587 ms
Wall time: 620 ms


In [9]:
df['ingredients'].iloc[0]

'winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt'

Explode the series containing list to a single list:

In [10]:
ingredients = df['ingredients'].explode().str.lower().str.strip()
print(f"Around {ingredients.shape[0]} ingredients are there in the dataset")

Around 231637 ingredients are there in the dataset


Now find the unique ingredients:

In [11]:
unique_ingredients = ingredients.unique()
print(f"There around {unique_ingredients.shape[0]} are unique out of {ingredients.shape[0]}")

There around 230475 are unique out of 231637


There might be some same words that mean the same but with different name.

Try with some keywords to see if it exists like search for 'egg' or 'milk'

In [12]:
keyword = 'bread'
unique_word = ingredients[ingredients.str.contains(keyword.lower())].unique()
if len(unique_word)> 0:
    sample = ", ".join(np.random.choice(unique_word,10))
    print(f"There are around {len(unique_word)} unique ingredients containing the work 'f{keyword}'.\n\n Example:\n\n {sample}....")
else:
    print(f"No ingredients for the keyword: {keyword}")

There are around 18673 unique ingredients containing the work 'fbread'.

 Example:

 water, white bread flour, wheat bread flour, dry milk, molasses, salt, chocolate chips, dried cherries, triple sec, orange rind, active dry yeast, crusty bread, plum tomatoes, fresh basil, extra virgin olive oil, garlic cloves, salt, dried rosemary, hot pepper flakes, white wine vinegar, chickpeas, garlic cloves, salt, fresh lemon juice, plain yogurt, tahini, pita breads, vegetable oil, cumin, butter, pine nuts, paprika, butter, oil, onions, sugar, flour, beef broth, water, salt & pepper, port wine, burgundy wine, french bread, mozzarella cheese, water, zucchini, ground round, garlic cloves, reduced-fat spaghetti sauce, salt, dried basil, dried oregano, fat-free cottage cheese, dried parsley, eggs, cooking spray, dry breadcrumbs, part-skim mozzarella cheese, fresh spinach, extra virgin olive oil, garlic cloves, white bread, red wine vinegar, cinnamon, paprika, ground cumin, cayenne pepper, salt & fresh

## Sentence encoder model


Selecting the encoder model: https://www.sbert.net/docs/pretrained_models.html

In [44]:
%%time
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

CPU times: user 1.04 s, sys: 263 ms, total: 1.31 s
Wall time: 42.4 s


In [45]:
# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

## Encoding the ingredients

In [46]:
%%time

## encode all the ingredients:
embedding = model.encode(df['ingredients'].values)

CPU times: user 1min 58s, sys: 2.01 s, total: 2min
Wall time: 1min 47s


## Exporting the embeddings into chunks

In [47]:
%%time
subarrays = np.split(embedding,7)
dir_path = r'/content/drive/MyDrive/Colab Notebooks/Projects/food_recommender/data_file'

for i, arr in tqdm(enumerate(subarrays), unit = " arrays"):

    filename = f'embed_arr_{i}.npy'
    file_path = os.path.join(dir_path, filename)

    with open(file_path, "wb") as f:
        np.save(f, arr)

0 arrays [00:00, ? arrays/s]

CPU times: user 16.7 ms, sys: 241 ms, total: 257 ms
Wall time: 1.25 s


In [48]:
df_final = df.copy()
df_final['embedding'] = list(zip(embedding))
df_final['embedding'] = df_final['embedding'].apply(lambda x: x[0])
df_final.head()

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),embedding
0,arriba baked winter squash mexican style,55,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,"[-0.008845049, 0.009866926, 0.028063796, 0.101..."
1,a bit different breakfast pizza,30,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,"[-0.054575536, 0.027983457, 0.065263726, 0.032..."
2,all in the kitchen chili,130,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"ground beef, yellow onions, diced tomatoes, to...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,"[-0.10401253, -0.024388006, 0.06653514, 0.0346..."
3,alouette potatoes,45,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","spreadable cheese with garlic and herbs, new p...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,"[-0.048682474, -0.04654153, -0.013404117, 0.01..."
4,amish tomato ketchup for canning,190,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"tomato juice, apple cider vinegar, sugar, salt...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,"[-0.06817169, 0.030255292, 0.026339032, 0.0327..."


In [49]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   name                 231636 non-null  object 
 1   minutes              231637 non-null  int64  
 2   n_steps              231637 non-null  int64  
 3   steps                231637 non-null  object 
 4   description          226658 non-null  object 
 5   ingredients          231637 non-null  object 
 6   n_ingredients        231637 non-null  int64  
 7   calories             231637 non-null  float64
 8   total fat (PDV)      231637 non-null  float64
 9   sugar (PDV)          231637 non-null  float64
 10  sodium (PDV)         231637 non-null  float64
 11  protein (PDV)        231637 non-null  float64
 12  saturated fat (PDV)  231637 non-null  float64
 13  carbohydrates (PDV)  231637 non-null  float64
 14  embedding            231637 non-null  object 
dtypes: float64(7), in

## Finally calculate the cosine similarity

In [51]:
def cosine_similarity(vec1,vec2):
    '''
    Returns the cosine similarity between two vectors of n dimension
    '''
    denom = np.sqrt(np.sum(np.square(vec1))) * np.sqrt(np.sum(np.square(vec2)))
    return np.round(np.dot(vec1,vec2) / denom * 100, 2)

In [52]:
index = 1
data = df_final.iloc[index]
recipe, vector = data['name'], data['embedding']

print("Name of the dish:", recipe)

Name of the dish: a bit different  breakfast pizza


In [53]:
%%time

df_result = df_final.copy()
df_result['similarity'] = df_result['embedding'].apply(lambda x : cosine_similarity(vector, x))

CPU times: user 10.7 s, sys: 697 ms, total: 11.4 s
Wall time: 11.1 s


In [54]:
df_result.sort_values(by="similarity", ascending=False).head(10)

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),embedding,similarity
1,a bit different breakfast pizza,30,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,"[-0.054575536, 0.027983457, 0.065263726, 0.032...",100.00
188135,skillet pizza,25,8,"['grease a 12 inch skillet', 'prepare pizza cr...",a different way and fun way to make pizza!,"prepared pizza crust, eggs, milk, pepper, baco...",7,331.1,42.0,2.0,21.0,34.0,54.0,0.0,"[-0.04533844, 0.012404476, 0.027594298, 0.0544...",89.83
12448,bacon cheese stromboli,27,6,"['on a ungreased baking sheet , roll the dough...",NaN,"prepared pizza crust, cheddar cheese, mozzarel...",6,303.3,37.0,12.0,44.0,29.0,57.0,2.0,"[-0.05913764, 0.013851965, 0.029617699, 0.0520...",86.96
94179,good morning breakfast pizza,25,9,"['preheat oven to 425', 'place ingredients on ...",i found this recipe online while looking for s...,"prepared pizza crust, cheese, hard-boiled eggs...",10,198.8,18.0,5.0,24.0,29.0,29.0,2.0,"[-0.052115414, -0.007797755, 0.040253118, 0.04...",86.73
112991,italian sausage onion pizza,25,9,"['heat oven to 400f', 'cook sausage in nonstic...",from french's website.,"sausage, bell pepper, pizza sauce, prepared pi...",6,241.3,27.0,18.0,30.0,23.0,37.0,2.0,"[-0.08934609, -0.011838251, 0.05330121, 0.0842...",86.17
75573,easy cheesy pizzas on the grill,17,13,"['grill directions: heat grill', 'cut eight 6 ...",from an old pillsbury cookbook. i suppose you...,"prepared pizza crust, pizza sauce, pepperoni, ...",6,204.2,22.0,7.0,20.0,26.0,36.0,1.0,"[-0.045810957, 0.011214065, 0.032459296, 0.036...",85.28
74217,e z pizza pockets,35,12,"['preheat oven to 375', 'unroll refrigerated c...",a quick no-mess meal that the kids love to hel...,"prepared pizza crust, pizza sauce, mozzarella ...",6,262.1,29.0,7.0,20.0,34.0,58.0,1.0,"[-0.057443466, 0.020050066, 0.010506704, 0.068...",85.01
5142,an aussie vegemite pizza,20,4,"['preheat oven to 190c', 'spread the vegemite ...",i like to zap the onion in the microwave with ...,"prepared pizza crust, vegemite, ham, onion, ta...",6,574.9,62.0,9.0,73.0,89.0,117.0,2.0,"[-0.019809524, 0.028295083, 0.06934482, 0.0175...",84.85
130629,margherita pizza,25,15,['preheat oven to specks on the boboli package...,a great summer pizza-light & easy to make. my ...,"prepared pizza crust, extra virgin olive oil, ...",10,376.3,47.0,13.0,35.0,39.0,60.0,2.0,"[-0.07858314, 0.0035144775, 0.05704327, 0.0470...",84.61
19185,bbq chicken pizza with feta,32,7,"['preheat oven to 450f', 'place pizza crust on...","from kraft foods, the combination of onion, bb...","prepared pizza crust, mozzarella cheese, grill...",6,240.2,21.0,11.0,27.0,43.0,42.0,1.0,"[-0.051184162, 0.016800782, 0.004887824, 0.082...",84.29
